<a href="https://colab.research.google.com/github/anvargasch/IEStoPRBT/blob/main/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clone the repository that contains the functions and the photometric sample files.

In [ ]:
!git clone https://github.com/anvargasch/IEStoPRBT.git

In [ ]:
from IEStoPRBT import readIES

# Example of reading an IES file and downloading the header and intensity matrix

In [ ]:
filename = '/content/IEStoPRBT/IESfiles/IESsiemens.ies'  # Replace with the path to your IES file
Lumen,fact,Num_vert_angle,Num_hori_angle,unid = readIES.IES_header(filename) # Function to return the phometrics parameters
print("Lumen:",Lumen,", Factor:", fact,", #Vert:",Num_vert_angle ,", #Horiz:",Num_hori_angle,", Units:", unid, "(1:feet, 2:meters)")

In [ ]:
filename = '/content/IEStoPRBT/IESfiles/IESsiemens.ies'# Replace with the path to your IES file
mat=readIES.IES_Intensity_matrix (filename) # Function to return the intensity matrix
print(mat)

In [ ]:
filename = '/content/IEStoPRBT/IESfiles/IESsiemens.ies' # Replace with the path to your IES file
img=readIES.IES2PBRT (filename)  #Function to return an image for use in PBRT

In [ ]:
import matplotlib.pyplot as plt   # Graphing functions
plt.figure(1)
plt.imshow(img ,cmap=plt.cm.gray), plt.title("Imagen IES")
plt.imshow

In [ ]:
readIES.save_EXR(img,'/content/IMG_EXR10.exr') #Function to save the PBRT image in EXR format